In [1]:
%load_ext autoreload
%autoreload 2

import sys
import json
sys.path.append('code/')

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import os, binascii, hashlib, base58

from parse_config import ConfigDict
from balances import Balances
from simulation import Simulation

pd.set_option('display.max_rows', 20)

config = ConfigDict().parse()

def print_params(params):
    for k,v in params().items():
        print("{}: {}".format(k,v))

## No initial issuance

In [2]:
b = Balances(**config['default'])
print_params(b.params)
print("\nGini Coefficient: {:0.2f}".format(b.gini()))
b.data.head()

n_users: 10000
n_miners_max: 100
init_issue: False
reward_dist: False
reward: 100.0
greed_factor: 0.05

Gini Coefficient: 0.00


,address,greedy,miner,values
0,5JTQxreLncjNYaZPEhaWao6XKVMQYyfFSba9StooVr1J3p...,False,False,0.0
1,5Jc8ECBLYV2w2Lanvqg3PTRVcm9Ed8zD9G1hW2y1pc9vzk...,False,False,0.0
2,5KZsf4Q6auTB9ub5sGovY3tu29561M83ATRs78p8ZarQGS...,False,False,0.0
3,5JxjCeY2a82avYwDC3gag9bAzs6PpyL52WnsnxMtDki58P...,False,False,0.0
4,5KV5hJ6P1iJNJ7GjHHA3mLeQ9Gss2XeT2ZZuvmNfYBhGHb...,False,False,0.0


## Fair initial issuance

In [3]:
b = Balances(**config['init_issue_fair_greedy'])
print_params(b.params)
print("\nGini Coefficient: {:0.2f}".format(b.gini()))
b.data.head()

n_users: 10000
n_miners_max: 100
init_issue: True
reward_dist: False
reward: 100.0
greed_factor: 0.05
init_issue_total: 1000000.0
init_issue_max: 500.0
init_issue_users: 9900

Gini Coefficient: 0.49


,address,greedy,miner,values
0,5K6ETxT1ztqxaedt78bM5wv4pBWa4aS5pw9NwycJT4R9wx...,False,False,14.788049
1,5Jfr6jWpbPPoyzbF5SCD4KF5ynG7rFtxU5ozNCGmFMYgG7...,False,False,23.711492
2,5K1Wf8kwMGQYKku94HEzKZKNfbr34JmeR9jxaYy41at7bX...,False,False,71.339273
3,5KNJU3cYXrNM4Y2KGnJgARZdKvkGJEWys7Yis4UHhxkPPD...,False,False,85.287527
4,5K9ivuU5it14mmWvZC1NnwzRZZtiVrGBCDHjxd7UhGBTUK...,False,False,32.297598


## Real initial issuance

In [4]:
b = Balances(**config['init_issue_real_greedy'])
print_params(b.params)
print("\nGini Coefficient: {:0.2f}".format(b.gini()))
b.data.head()

n_users: 10000
n_miners_max: 100
init_issue: True
reward_dist: False
reward: 100.0
greed_factor: 0.05
init_issue_total: 1000000.0
init_issue_max: 50000.0
init_issue_users: 1000

Gini Coefficient: 0.95


,address,greedy,miner,values
0,5KaRdsbPz1KNebBNKFUQyiQuC2iT1SV1w1Zi6RLaM1UZfo...,False,False,0.0
1,5HuH4kVb4JWzKmBad29SgKdKpmq2aeq5i5m1Awd1oUTNLV...,False,False,0.0
2,5KhyLq7af56Fz2fVc8yb1hBwBUdTx1DtR9koVFEBK7LNQ5...,False,False,0.0
3,5HxcxMnbuMfoTs7m4S6MLWEUsKCpSo8hN2o1D2hhnVGYDU...,False,False,0.0
4,5KSeZPTFXxTU69doY1NrHhaEGbbwdvrqYnD254U1bkyEgM...,False,False,0.0


## Simulation experiments on greed

In [ ]:
config = ConfigDict().parse()
fig, axs = plt.subplots(2,3)
exp_states = {'default': "No initial issuance, with greed",
              'no_greed': "No initial issuance, no greed",
              'init_issue_fair_greedy': "Fair initial issuance, with greed",
              'init_issue_fair_alt': "Fair inital issuance, no greed",
              'init_issue_real_greedy': "Real initial issuance, with greed", 
              'init_issue_real_alt' : "Fair initial issuance, no greed" }
g = []
g_init = []
for exp_key in exp_states.keys():
    s = Simulation(balances=Balances(**config[exp_key]))
    g_data = s.simulate()
    g.append(g_data)
    g_init.append(s.balances.gini(initial=True))
    print("{}\t\t{:.3f}\t\t{:.3f}".format(exp_states[exp_key], s.balances.gini(initial=True), np.mean(g_data)))

No initial issuance, with greed		0.000		0.234
No initial issuance, no greed		0.000		0.151
Fair initial issuance, with greed		0.495		0.247


In [ ]:
for x, ax in zip(g, axs):
    ax.hist(g= bins=np.arange(0.,1.,0.05))